**Оглавление:**
* [Реализация векторизатора](#tf-idf-model)
* [Реализация языковой модели](#Language-model)
* [Выдача векторной модели](#Выдача-векторной-модели)
* [Выдача языковой модели](#Выдача-языковой-модели)

In [1]:
import wikipedia
import nltk
import re
from pymystem3 import Mystem
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/a.tsigankov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/a.tsigankov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# for pretty output
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [3]:
pd.set_option('max_colwidth', 80)
wikipedia.set_lang('ru')
tokenizer = nltk.RegexpTokenizer(r"\w+")
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

Факты

In [4]:
facts = [
    "Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправдан и выпущен на свободу.",
    "Историк утверждает, что, прежде чем допустить крестьян на личный прием к вождю мирового пролетариата, их проводили через дезинфекционный пункт.",
    "В России акции протеста проходят не только на площадях, но и на поездах."
]

Название статей Википедии, на которые ссылаются факты

In [5]:
wikipedia_pages = {
    1: ["Шмидт, Генрих (врач)"],
    2: ["Ходоки у В. И. Ленина", "Дезинфекция"],
    3: ["Проезд снаружи поездов"]
}

### Скачивание и обработка статей из Википедии

In [6]:
def get_page_text(pagename):
    page = wikipedia.page(pagename)
    return page.content

In [7]:
def remove_trash(page_text):
    page_text = re.sub('\s', ' ', page_text)
    page_text = re.sub('[А-Я]\.', '', page_text)
    page_text = re.sub(r'\=\= Примечания[\w\s\=]*', '', page_text)
    page_text = page_text.replace('.', '. ')
    page_text = page_text.replace('\n', ' ')
    headers = re.findall(r"\=\=.*?\=\=", page_text)
    for header in headers:
        page_text = page_text.replace(header, '')
    
    return page_text

In [8]:
documents = []

for num in wikipedia_pages.keys():
    for page in wikipedia_pages[num]:
        page_text = remove_trash(get_page_text(page))
        sentences = nltk.sent_tokenize(page_text, language="russian")
        documents.extend(sentences)

In [9]:
# добавление фактов
documents.extend(facts)

In [10]:
wiki = pd.DataFrame(documents)

In [11]:
wiki.columns = ['document']

In [12]:
wiki['relevance'] = 0

In [13]:
wiki.at[0, 'relevance'] = 1
wiki.at[234, 'relevance'] = 1
wiki.at[59, 'relevance'] = 1
wiki.at[61, 'relevance'] = 1
wiki.at[62, 'relevance'] = 1

In [14]:
wiki.at[10, 'relevance'] = 2
wiki.at[13, 'relevance'] = 2
wiki.at[60, 'relevance'] = 2
wiki.at[345, 'relevance'] = 2

In [15]:
wiki.head()

,document,relevance
0,"Эрнст Генрих Шмидт (нем. Ernst Heinrich Schmidt; 27 марта 1912, Альтенбург,...",1
1,Генрих Шмидт родился 27 марта 1912 года в Альтенбурге.,0
2,В 1933 году был зачислен в СС (№ 23 069).,0
3,В 1937 году получил степень доктора медицинских наук в Лейпцигском университ...,0
4,После начала Второй мировой войны работал в лазарете Ваффен С С 1941 года с...,0


### Лемматизация 

In [16]:
def lemmatize(sentence):
    return ''.join((mystem.lemmatize(' '.join(tokenizer.tokenize(sentence))))).replace('\n', '')

In [17]:
def remove_stop_words(sentence):
    return ' '.join([word for word in sentence.split(' ') if word not in russian_stopwords])

In [18]:
wiki['lemmatized_document'] = wiki.document.apply(lemmatize)
wiki['lemmatized_document'] = wiki.lemmatized_document.apply(remove_stop_words)

In [19]:
wiki['length'] = wiki.lemmatized_document.apply(lambda x: len(x.split()))

In [20]:
wiki = wiki[wiki.length > 1].reset_index()

In [21]:
# wiki.to_csv('dump.csv')

In [22]:
wiki.tail()

,index,document,relevance,lemmatized_document,length
540,561,Дата обращения: 7 декабря 2012.,0,дата обращение 7 декабрь 2012,5
541,562,Архивировано 27 декабря 2012 года.,0,архивировать 27 декабрь 2012 год,5
542,563,"Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправда...",0,нацистский врач эсэсовец работать шесть концлагерь дважды оправдывать выпуск...,10
543,564,"Историк утверждает, что, прежде чем допустить крестьян на личный прием к вож...",0,историк утверждать прежде допускать крестьянин личный прием вождь мировой пр...,13
544,565,"В России акции протеста проходят не только на площадях, но и на поездах.",0,россия акция протест проходить площадь поезд,6


#### tf-idf model

In [23]:
class Vectorizer:
    
    def __init__(self, corpus):
        self.corpus = corpus
        self.vocabulary = np.unique([word for word in ' '.join(corpus).split(' ') if len(word) != 0])
        self.corpus_length = len(corpus)
        self.vocabulary_length = len(self.vocabulary)
        self.vectorized_idf = np.vectorize(lambda x: np.log10(self.corpus_length / x))
        self.inverse_vocabulary = {v: k for k, v in dict(enumerate(self.vocabulary)).items()}
        self.vectorize()
        
    def vectorize(self, tf_normed=False):
        tf = []
        df = np.zeros(self.vocabulary_length, dtype=int)

        for document in self.corpus:
            document_vector = np.zeros(self.vocabulary_length, dtype=int)
            for word in document.split(' '):
                if word != '':
                    document_vector[self.inverse_vocabulary[word]] += 1
            for word in np.unique(document.split(' ')):
                if word != '':
                    df[self.inverse_vocabulary[word]] += 1
            tf.append(document_vector)

        tf = np.stack(tf, axis=0)
        
        self.tf = tf
        
        if tf_normed:
            
            def tf_max(array):
                array = array / array.max()
                array[array != 0] *= 0.6
                array[array != 0] += 0.4
                return array

            tf = np.apply_along_axis(tf_max, 1, tf)

        idf = self.vectorized_idf(df)
        tf_idf = np.apply_along_axis(lambda x: x * idf, 1, tf)
        
        return tf_idf
    
    def get_inverse_vocabulary(self):
        return self.inverse_vocabulary
    
    def get_tf(self):
        return self.tf

#### Language model

In [24]:
class LanguageModel:
    def __init__(self, tf):
        self.tf = tf
        
    def p(self, term):
        return np.sum(self.tf[:, term]) / np.sum(self.tf)
    
    def p_d(self, term, document):
        return self.tf[document, term] / np.sum(self.tf[document])
    
    def calculate(self, document, param=0.5):
        r = []
        for i in range(len(self.tf)):
            pp = 1
            for j in range(len(self.tf[document])):
                if self.tf[document][j] != 0:
                    pp *= ((1 - param) * self.p(j) + param * self.p_d(j, i))
            r.append(pp)
        return np.argsort(np.array(r))[-11:][::-1]

In [25]:
vectorizer = Vectorizer(wiki['lemmatized_document'])
lang_model = LanguageModel(vectorizer.get_tf())

In [26]:
tfidf_weights = vectorizer.vectorize()
tfidf_weights_normed = vectorizer.vectorize(tf_normed=True)

In [27]:
facts_indexes = [542, 543, 544]

In [28]:
similarity = cosine_similarity(tfidf_weights, tfidf_weights)
similarity_normed = cosine_similarity(tfidf_weights_normed, tfidf_weights_normed)

In [29]:
def get_nearest_documents(similarity, index):
    
    indices = np.argsort(similarity[index])[-11:][::-1]
    cosine = similarity[index][indices]
    
    return cosine, wiki.loc[indices]['document'].values, wiki.loc[indices]['relevance'].values

def print_results_for_document(similarity, index):
    
    cosines, documents, relevances = get_nearest_documents(similarity, index)
    
    fact = documents[0]
    
    printmd("Факт: " + "**" + str(fact) + "**")
    printmd("Близкие документы:")
    
    for doc, cosine, relevance in zip(documents[1:], cosines[1:], relevances[1:]):
        printmd(f"* [cos: {cosine:.2f}][rel: {relevance}] {doc}")

In [30]:
def results_for_document_lang(index, param=0.5):
    
    # cosines, documents, relevances = get_nearest_documents(similarity, index)
    indices = lang_model.calculate(index, param)
    documents = wiki.loc[indices[0:]]['document'].values
    relevances = wiki.loc[indices[0:]]['relevance'].values
    
    fact = documents[0]
    
    printmd("Факт: " + "**" + str(fact) + "**")
    printmd("Близкие документы:")
    
    for doc, relevance in zip(documents[1:], relevances[1:]):
        printmd(f"* [rel: {relevance}] {doc}")

# Выдача векторной модели

In [31]:
printmd('## Без нормализации tf')
print_results_for_document(similarity, facts_indexes[0])
printmd('## С нормализацией tf')
print_results_for_document(similarity_normed, facts_indexes[0])

## Без нормализации tf

Факт: **Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправдан и выпущен на свободу.**

Близкие документы:

* [cos: 0.20][rel: 2] В итоге он был оправдан.

* [cos: 0.20][rel: 0] После начала Второй мировой войны работал в лазарете Ваффен С  С 1941 года служил врачом в концлагере Бухенвальд, в июне 1942 года был переведён в концлагерь Майданек.

...

* [cos: 0.08][rel: 1] Эрнст Генрих Шмидт (нем.  Ernst Heinrich Schmidt; 27 марта 1912, Альтенбург, Германская империя — 28 ноября 2000, Целле, Германия) — немецкий врач, гауптштурмфюрер СС, служивший в различных концлагерях.

* [cos: 0.08][rel: 2] Из-за отсутствия доказательств 20 марта 1979 года Шмидт вновь был оправдан, а 19 апреля 1979 года освобождён.

* [cos: 0.08][rel: 0] К 1950 году, уже много лет работая над образом   Ленина, художник тем не менее не получал полного удовлетворения от результатов своего труда.

* [cos: 0.08][rel: 0] С некоторых пор и в России они получили распространение и стали атрибутом не только кабинета врача или аптечки автомобилиста или врача скорой помощи, но и обычной домашней аптечки.

* [cos: 0.07][rel: 0] В 1950 году на Ленинградском фарфоровом заводе была выпущена декоративная ваза с росписью художника Валентина Сарлунда, в основе которой лежала картина Владимира Серова.

In [32]:
printmd('## Без нормализации tf')
print_results_for_document(similarity, facts_indexes[1])
printmd('## С нормализацией tf')
print_results_for_document(similarity_normed, facts_indexes[1])

## Без нормализации tf

Факт: **Историк утверждает, что, прежде чем допустить крестьян на личный прием к вождю мирового пролетариата, их проводили через дезинфекционный пункт.**

Близкие документы:

* [cos: 0.13][rel: 2] Например, Нина Тумаркин упоминала, что в Кремле был организован дезинфекционный пункт, через который пропускали посетителей председателя Совнаркома.

* [cos: 0.11][rel: 0] Паровоздушная смесь является действующим началом в пароформалиновой дезинфекционной камере; в дезинфекционных камерах обеззараживают вещи больного и постельные принадлежности.

...

* [cos: 0.11][rel: 0] Советский, а после эмиграции — британский историк искусства Игорь Голомшток утверждал, что «обязательной принадлежностью колхозных клубов и сельсоветов было изображение типа „Ходоки у Ленина“  Серова».

* [cos: 0.10][rel: 0] Крестьянин, затаив дыхание, «с надеждой и любовью безотрывно смотрит на вождя».

* [cos: 0.10][rel: 0] В этой обстановке в деревне возникали сложные проблемы, вызывавшие потребность личного общения крестьян с председателем рабоче-крестьянского правительства.

* [cos: 0.09][rel: 0] Она утверждала, что «Ильич был особенно обходителен с крестьянами-ходоками» и «как-то особенно гостеприимно с ними здоровался при встрече».

* [cos: 0.09][rel: 0] Паровоздушная смесь является действующим началом в пароформалиновой дезинфекционной камере; в дезинфекционных камерах обеззараживают вещи больного и постельные принадлежности.

In [33]:
printmd('## Без нормализации tf')
print_results_for_document(similarity, facts_indexes[2])
printmd('## С нормализацией tf')
print_results_for_document(similarity_normed, facts_indexes[2])

## Без нормализации tf

Факт: **В России акции протеста проходят не только на площадях, но и на поездах.**

Близкие документы:

* [cos: 0.17][rel: 2] Одна такая акция имела место 30 июля 2010 года на Горьковском направлении МЖД в день запуска скоростного поезда «Сапсан» сообщением Москва — Нижний Новгород, одной из заявленных целей которой, помимо приветствия «Сапсана», являлся протест против отмены пригородных электричек, обусловленной ремонтом путей в преддверии запуска скоростного поезда).

* [cos: 0.17][rel: 0] Глядя на них, художник думал о дорогах, по которым они прошли.

...

* [cos: 0.06][rel: 0] Размер действующего по состоянию на 2010-е годы в России штрафа за проезд снаружи поезда, предусмотренного частью 1 статьи 11.

* [cos: 0.06][rel: 0] Также в России с 2010 года участились случаи проезда снаружи поездов в метрополитенах, а в 2011 года зафиксирована попытка подобной поездки на скоростном поезде «Сапсан».

* [cos: 0.06][rel: 0] Санкции в России  По состоянию на 2019 год в России проезд снаружи транспортных средств расценивается как административное правонарушение, санкции за которое различаются в зависимости от вида транспорта.

* [cos: 0.06][rel: 0] Также в связи с тем, что на ряде железных дорог среди трейнсёрферов время от времени могут появляться лица, которые при проезде занимаются вандализмом и наносят вред подвижному составу или создают помехи в движении поездов (например, рисуют граффити на лобовых стёклах кабин и корпусах поездов, ломают дворники и разбивают стёкла и фары, открывают концевые краны тормозной или напорной магистрали с целью экстренной остановки поезда без необходимости, опускают токоприёмники с целью обесточить поезд, бросают предметы в проходящие поезда и т.  д. ), ряд машинистов и железнодорожных любителей имеют резко негативное отношение ко всем пассажирам, проезжающим снаружи поездов, рассматривая каждого из них как потенциального вредителя.

* [cos: 0.05][rel: 0] В ряде стран, где существуют развитые сообщества трейнсёрферов, в том числе и в ряде регионов России, ими могут устраиваться массовые мероприятия по катанию снаружи поездов группами до нескольких десятков человек.

# Выдача языковой модели

In [34]:
results_for_document_lang(facts_indexes[0])

Факт: **Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправдан и выпущен на свободу.**

Близкие документы:

* [rel: 0] После начала Второй мировой войны работал в лазарете Ваффен С  С 1941 года служил врачом в концлагере Бухенвальд, в июне 1942 года был переведён в концлагерь Майданек.

* [rel: 0] В октябре 1943 года стал главным врачом в концлагере Гросс-Розен, а с сентября 1944 года служил в концлагере Дахау.

* [rel: 1] Эрнст Генрих Шмидт (нем.  Ernst Heinrich Schmidt; 27 марта 1912, Альтенбург, Германская империя — 28 ноября 2000, Целле, Германия) — немецкий врач, гауптштурмфюрер СС, служивший в различных концлагерях.

...

* [rel: 2] Из-за отсутствия доказательств 20 марта 1979 года Шмидт вновь был оправдан, а 19 апреля 1979 года освобождён.

* [rel: 0] Предметы, жесты и позы персонажей работают на главную идею произведения.

* [rel: 0] В 1950 году на Ленинградском фарфоровом заводе была выпущена декоративная ваза с росписью художника Валентина Сарлунда, в основе которой лежала картина Владимира Серова.

* [rel: 0] С некоторых пор и в России они получили распространение и стали атрибутом не только кабинета врача или аптечки автомобилиста или врача скорой помощи, но и обычной домашней аптечки.

* [rel: 0] К 1950 году, уже много лет работая над образом   Ленина, художник тем не менее не получал полного удовлетворения от результатов своего труда.

In [35]:
results_for_document_lang(facts_indexes[1])

Факт: **Историк утверждает, что, прежде чем допустить крестьян на личный прием к вождю мирового пролетариата, их проводили через дезинфекционный пункт.**

Близкие документы:

* [rel: 2] Например, Нина Тумаркин упоминала, что в Кремле был организован дезинфекционный пункт, через который пропускали посетителей председателя Совнаркома.

* [rel: 0] Советский, а после эмиграции — британский историк искусства Игорь Голомшток утверждал, что «обязательной принадлежностью колхозных клубов и сельсоветов было изображение типа „Ходоки у Ленина“  Серова».

* [rel: 0] Сам Серов утверждал, что этот персонаж самовольно оставил окопы мировой войны, фронт стал для него настоящей революционной школой, к Ленину он пришёл, будучи избран для этой миссии своими земляками.

...

* [rel: 0] В этой обстановке в деревне возникали сложные проблемы, вызывавшие потребность личного общения крестьян с председателем рабоче-крестьянского правительства.

* [rel: 0] Она утверждала, что «Ильич был особенно обходителен с крестьянами-ходоками» и «как-то особенно гостеприимно с ними здоровался при встрече».

* [rel: 0] После того, как   Калинин был избран председателем ВЦИКа, Владимир Ильич передал ему личное общение с крестьянами, так как «Михаил Иванович лучше других знает деревню и поэтому он спокоен за то, что каждое ходатайство крестьян будет рассмотрено подробно и хорошо».

* [rel: 0] Кандидат искусствоведения Раиса Аболина считала, что художник в работе над образом Ленина отталкивался от цитаты из воспоминаний секретаря Совнаркома РСФСР и личного секретаря Ленина   Горбунова: «он садился против крестьянина так близко, что колени их соприкасались, ласково улыбаясь, нагибался немного вперёд, как бы прислушиваясь, и деловито выспрашивал, выпытывал, давал указания».

* [rel: 0] Художник реконструировал мельчайшие детали эпохи, но точность деталей и глубокая психологическая характеристика персонажей лишь оттеняли, с точки зрения официального искусствоведения того времени, «создание атмосферы волнующей сердечности отношений между Лениным и простыми крестьянами, глубокого чувства уважения вождя к этим, быть может, даже неграмотным людям».

In [36]:
results_for_document_lang(facts_indexes[2])

Факт: **В России акции протеста проходят не только на площадях, но и на поездах.**

Близкие документы:

* [rel: 2] Одна такая акция имела место 30 июля 2010 года на Горьковском направлении МЖД в день запуска скоростного поезда «Сапсан» сообщением Москва — Нижний Новгород, одной из заявленных целей которой, помимо приветствия «Сапсана», являлся протест против отмены пригородных электричек, обусловленной ремонтом путей в преддверии запуска скоростного поезда).

* [rel: 0] Глядя на них, художник думал о дорогах, по которым они прошли.

* [rel: 0] Также в связи с тем, что на ряде железных дорог среди трейнсёрферов время от времени могут появляться лица, которые при проезде занимаются вандализмом и наносят вред подвижному составу или создают помехи в движении поездов (например, рисуют граффити на лобовых стёклах кабин и корпусах поездов, ломают дворники и разбивают стёкла и фары, открывают концевые краны тормозной или напорной магистрали с целью экстренной остановки поезда без необходимости, опускают токоприёмники с целью обесточить поезд, бросают предметы в проходящие поезда и т.  д. ), ряд машинистов и железнодорожных любителей имеют резко негативное отношение ко всем пассажирам, проезжающим снаружи поездов, рассматривая каждого из них как потенциального вредителя.

...

* [rel: 0] Доктор исторических наук, помощник и секретарь   Ленина Владимир Бонч-Бруевич вспоминал, что первая встреча Ленина с ходоками продолжалась пять часов, а всего через общение с ним за несколько месяцев прошло более 6000 человек.

* [rel: 0] Размер действующего по состоянию на 2010-е годы в России штрафа за проезд снаружи поезда, предусмотренного частью 1 статьи 11.

* [rel: 0] В Европе по состоянию на конец 2000-х — начало 2010-х годов проезд снаружи поездов наибольшее распространение получил в России в Московской и Ленинградской области и на Украине в Киевской области, в особенности на пригородных поездах.

* [rel: 0] Транспортная полиция России задерживает зацеперов.

* [rel: 0] В ряде стран, где существуют развитые сообщества трейнсёрферов, в том числе и в ряде регионов России, ими могут устраиваться массовые мероприятия по катанию снаружи поездов группами до нескольких десятков человек.